# Example notebook
## How to incorporate/use code from `common` module

### Import functions from common module to use where relevant

In [ ]:
from common import load_image, show_image, load_data, split_data, validation_scores, merge_results

### Simple example usage of each function

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline

# Load data
df = load_data()

# Split data
X_train, X_test, y_train, y_test = split_data(df)
# Preview shape of split data
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
## Simple example of running an experiment
# Create a pipeline
pipe = make_pipeline(
    StandardScaler(),
    SVC(random_state=0, probability=True)
)
# Define parameters and options
param_grid = {
    'svc__C': [0.1, 1]
}
# Get results for grid search with each of the 3 required scoring metrics (detailed progress updates with verbose=3)
results = validation_scores(pipe, param_grid, X_train, y_train, verbose=3)
# Take a look at the validation results
print('Validation results:')
print(results)

# Merge results into a unified dataframe (scorer will be new column)
merged_df = merge_results(results)

# Save merged results to CSV file
merged_df.to_csv('example_results.csv', index=False)


In [ ]:
# Load an image by providing the image name and class (class name)
img = load_image(df.iloc[0]['image_name'], df.iloc[0]['class'])
# Show the image
show_image(img)